### Requerimientos

In [1]:
import numpy as np
import pandas as pd
import csv
from scipy.stats import wilcoxon 
from statistics import mean

### Definición del tipo de test (maximización o minimización)

In [2]:
def mejor(x,y):
    return x>y # maximización
#    return x<y # minimización

def peor(x,y):
    return x<y # maximización
#    return x>y # minimización

### Definición de la clase Ranking

In [3]:
class Ranking:
    def __init__(self, name):
        self.name = name # Nombre del algoritmo
        self.wins = 0 # Veces que ha ganado
        self.losses = 0 # Veces que ha perdido
    def __lt__(self,x):
        return (self.wins-self.losses) < (x.wins-x.losses)
    def __str__(self):
        return f"{self.name:<15} {self.wins:>10} {self.losses:>10} {self.wins - self.losses:>15}"

### Calcula la matriz de wins - losses de las medias a partir de los scores de un problema

In [4]:
def CalculateWinsLossesMatrixMean(scores):
    labels = scores.columns.values
    nScores = len(labels)
    WinLossMatriz = np.zeros((nScores,nScores)) 
    for i in range(nScores-1):            
        score_i = scores.iloc[:,i].values
        for j in range(i+1,nScores):
            score_j = scores.iloc[:,j].values
            if mejor(mean(score_i),mean(score_j)):
                WinLossMatriz[i,j] = 1
                WinLossMatriz[j,i] = -1
            if peor(mean(score_i),mean(score_j)):
                WinLossMatriz[i,j] = -1
                WinLossMatriz[j,i] = 1
    return WinLossMatriz

### Calcula la matriz de wins - losses de las diferencias estadísticamente significativas para un test estadistico dado, junto con la matriz de p-values, a partir de los scores de un problema

In [5]:
def CalculateWinsLossesMatrixStat(scores,stat):
    labels = scores.columns.values
    nScores = len(labels)
    WinLossMatriz = np.zeros((nScores,nScores)) 
    pValues = np.zeros((nScores,nScores)) 
    for i in range(nScores-1):            
        score_i = scores.iloc[:,i].values
        for j in range(i+1,nScores):
            score_j = scores.iloc[:,j].values
            if not all(x_i == y_i for x_i, y_i in zip(score_i, score_j)): # si son datos diferentes
                _, p_value = stat(score_i, score_j)  
                pValues[i,j] = p_value
                pValues[j,i] = p_value                
                if p_value<0.05:
                    if mejor(mean(score_i),mean(score_j)):
                        WinLossMatriz[i,j] = 1
                        WinLossMatriz[j,i] = -1                        
                    if peor(mean(score_i),mean(score_j)):
                        WinLossMatriz[i,j] = -1
                        WinLossMatriz[j,i] = 1                        
    return WinLossMatriz,pValues

### Calcula las veces que cada algoritmo ha ganado y ha perdido a partir de la matriz de wins - losses de un problema (aplicable tanto para las medias como para los test estadísticos)

In [6]:
def CalculateWinsLossesAmount(WinsLossesMatriz,labels):
    nScores = len(labels)
    WinsLossesAmount = [Ranking(scoreName) for scoreName in labels]
    for i in range(nScores-1):            
        for j in range(i+1,nScores):
            if WinsLossesMatriz[i,j]==1:            
                WinsLossesAmount[i].wins += 1
                WinsLossesAmount[j].losses += 1
            if WinsLossesMatriz[i,j]==-1:            
                WinsLossesAmount[j].wins += 1
                WinsLossesAmount[i].losses += 1
    return WinsLossesAmount

### Acumula las cantidades de wins - losses de un problema en la lista de cantidades wins - losses totales (aplicable tanto para las medias como para los test estadísticos)

In [7]:
def AddWinsLossesAmount(WinsLossesTotalAmount,WinsLossesAmount):
    for i in range(len(WinsLossesAmount)):
        WinsLossesTotalAmount[i].wins += WinsLossesAmount[i].wins
        WinsLossesTotalAmount[i].losses += WinsLossesAmount[i].losses

### Impresión de la matriz wins - losses (aplicable tanto para las medias como para los test estadísticos)

In [8]:
def PrintMatriz(WinLossMatriz,labels):
    print("win: El algoritmo en la columna gana al algoritmo de la fila")
    print("loss: El algoritmo en la columna pierde frente al algoritmo de la fila")
    print("tie: El algoritmo en la columna empata con al algoritmo de la fila")
    n = len(labels)
    col_width = 10  # Ancho fijo para cada columna
    # Imprimir encabezados de columna
    print(" " * col_width, end="")
    for j in range(n):
        print(f"{labels[j]:>{col_width}}", end="")
    print()
    for i in range(n):
        print(f"{labels[i]:<{col_width}}", end="")
        for j in range(n):
            if i == j:
                print(f"{'-':>{col_width}}", end="")
            elif WinLossMatriz[j,i] == 1:
                print(f"{'win':>{col_width}}", end="")
            elif WinLossMatriz[j,i] == -1:
                print(f"{'loss':>{col_width}}", end="")
            else:
                print(f"{'tie':>{col_width}}", end="")
        print()

### Impresión  del ranking wins - losses (aplicable tanto para las medias como para los test estadísticos)

In [9]:
def PrintRanking(WinLoss):
    Ranking = sorted(WinLoss, reverse=True)    
    print(f"{'Ranking':<15} {'Wins':>10} {'Losses':>10} {'Wins-Losses':>15}")
    for r in Ranking:
        print(r)

### Impresión de la matriz de p-values

In [10]:
def PrintPValuesMatriz(pValues, labels):
    n = len(labels)
    col_width = 15  # Ancho fijo para cada columna, ajusta según sea necesario
    print("p-values")
    # Imprimir encabezados de columna
    print(" " * col_width, end="")
    for j in range(n):
        print(f"{labels[j]:>{col_width}}", end="")
    print()

    # Imprimir filas con datos
    for i in range(n):
        print(f"{labels[i]:<{col_width}}", end="")
        for j in range(n):
            if i == j:
                print(f"{'-':>{col_width}}", end="")
            else:
                print(f"{pValues[i,j]:>{col_width}.8f}", end="")
        print()

### Realización de los test estadísticos con Wilcoxon Signed-Rank

In [16]:
import os


#COMPARAR los ea normal, ea weighted, ea weigthed con pesos a partir de rmse, voting normal, voting pesos calculados y los base luego podemos meter wrapped a todos

# 1. Configuración de archivos y rutas
dataset_names = ["Boston", "Concrete", "US_Crime", "Abalone", "Elevators"]
# CAMBIO: Apuntamos a la carpeta del weighted, cámbialo si quieres analizar la otra
base_path = "../results/regression-weighted/"

# 2. Inicialización (Leemos el primero para coger los nombres de los algoritmos)
first_file_path = os.path.join(base_path, dataset_names[0] + "_results.csv")

if os.path.exists(first_file_path):
    df_first = pd.read_csv(first_file_path)
    # FILTRADO DE COLUMNAS:
    # Cogemos las que tienen RMSE, pero QUITAMOS 'Val', 'Best_Base' y columnas de pesos/metadata
    all_rmse = [col for col in df_first.columns if "RMSE" in col]
    labels = [c for c in all_rmse if c not in ['RMSE_EA_Val', 'RMSE_Best_Base']]
    print(f"⚔️ Algoritmos compitiendo: {labels}")
else:
    print("❌ Error: No se encuentra el primer archivo.")
    labels = []

# Inicializamos el objeto Ranking global para Signed Rank (Wilcoxon)
WinsLossesTotalAmountSignedRank = [Ranking(scoreName) for scoreName in labels]

# 3. Bucle por cada Dataset
for name in dataset_names:
    file_path = os.path.join(base_path, name + "_results.csv")

    if not os.path.exists(file_path):
        print(f"⚠️ Saltando {name}, archivo no encontrado.")
        continue

    # Cargar datos crudos
    raw_data = pd.read_csv(file_path)
    print(f"\n{'='*60}")
    print(f"Dataset: {name}")
    print('='*60)

    # --- LIMPIEZA DE FILAS: Eliminamos la fila de medias ---
    # Puede ser 'Fold' == '-' o 'Fold' == 'MEDIA'
    fold_col = raw_data['Fold'].astype(str)
    raw_data = raw_data[~fold_col.isin(['-', 'MEDIA'])]

    print(f"✅ Filas válidas tras filtrar medias: {len(raw_data)}")

    # 1. Seleccionamos solo las columnas de los algoritmos
    scores = raw_data[labels]

    # 2. TRUCO MATEMÁTICO PARA RMSE (Minimización):
    # Multiplicamos por -1 para que "Mayor sea Mejor" y el Ranking funcione.
    scores = scores * -1

    # --- WINS-LOSSES Y P-VALUES DE SIGNED-RANK (Wilcoxon) ---
    WinsLossesMatrizSignedRank, pValuesSignedRank = CalculateWinsLossesMatrixStat(scores, wilcoxon)
    WinsLossesAmountSignedRank = CalculateWinsLossesAmount(WinsLossesMatrizSignedRank, labels)

    # Acumulamos al global
    AddWinsLossesAmount(WinsLossesTotalAmountSignedRank, WinsLossesAmountSignedRank)

    print("\n--- Signed Rank (Wilcoxon p<0.05) ---")
    PrintMatriz(WinsLossesMatrizSignedRank, labels)
    print('\n')
    PrintRanking(WinsLossesAmountSignedRank)


print("\n" + "="*60)
print("📊 RESULTADOS TOTALES ACUMULADOS")
print("   (Basado en Wilcoxon Signed-Rank)")
print("="*60 + "\n")

PrintRanking(WinsLossesTotalAmountSignedRank)
print('\n')

⚔️ Algoritmos compitiendo: ['RMSE_EA_Test', 'RMSE_ET', 'RMSE_RIDGE', 'RMSE_KNN']

Dataset: Boston
✅ Filas válidas tras filtrar medias: 10

--- Signed Rank (Wilcoxon p<0.05) ---
win: El algoritmo en la columna gana al algoritmo de la fila
loss: El algoritmo en la columna pierde frente al algoritmo de la fila
tie: El algoritmo en la columna empata con al algoritmo de la fila
          RMSE_EA_Test   RMSE_ETRMSE_RIDGE  RMSE_KNN
RMSE_EA_Test         -      loss      loss       tie
RMSE_ET          win         -       win       win
RMSE_RIDGE       win      loss         -       tie
RMSE_KNN         tie      loss       tie         -


Ranking               Wins     Losses     Wins-Losses
RMSE_EA_Test             2          0               2
RMSE_KNN                 1          0               1
RMSE_RIDGE               1          1               0
RMSE_ET                  0          3              -3

Dataset: Concrete
✅ Filas válidas tras filtrar medias: 10

--- Signed Rank (Wilcoxon p<0.05)

Los warning se deben a que los p-values son extremedamente pequeños. El cambio a una aproximación normal ocurre porque, en pruebas exactas, como la prueba de Wilcoxon o la de rangos con signo, el cálculo del valor p se basa en la distribución exacta de los datos. Este tipo de pruebas asume ciertas condiciones en la distribución de los datos para que el cálculo sea preciso. Sin embargo, cuando los valores p resultan extremadamente pequeños, puede indicar que las diferencias entre grupos son demasiado grandes, lo que altera la precisión y estabilidad del cálculo exacto.

Al tener valores p tan bajos, el test puede asumir que el cálculo exacto no es viable o que sus resultados podrían ser inestables, por lo que automáticamente opta por una aproximación normal. Esta aproximación permite estimar el valor p basándose en una distribución normal, que es robusta y adecuada para conjuntos de datos grandes o con diferencias pronunciadas, y simplifica los cálculos en esos casos extremos.

Este cambio, entonces, asegura estabilidad y eficiencia en los resultados cuando las diferencias observadas son significativas y el valor p exacto cae en el rango de números extremadamente pequeños.